<subject>
Homework How-To
</subject>

<details>

**Naming Conventions**

When naming your files for upload, you must follow the format below:

    <uni>_<assignment>_<details [optional]>.<extension>

For example, if I needed to hand in HW 0, any of the below formats would be sufficient for a file name:

- pl2648_hw0.txt
- pl2648_hw0.sh
- pl2648_hw0_all_in_one.txt
- Pl2648_hw0_bash_program.sh

This naming format allows for autograding of all assignments. If your files are not named with this format, you should expect a grade of zero for the assignment.

**Grading**

Possible points on late homeworks are deducted by 50% for each day they are late. For example, if you get 80% of the total possible credits on a homework but hand in that homework a day late, you would get 40%. Assignments two days late get zero points.

Once solutions are posted and graded assignments are handed back, students have 1 week to bring their grading discrepancies to a CA for consideration of possible grading errors. 

**Getting Help**

Asking for help is a great way to increase your chance of success. However there are some rules. When asking for help (especially from a fellow student), *you can show your helper your code but you can not view theirs*. You work needs to be your own. 

If you need to reach out to a CA or the Professor for help, please do so via Piazza and not via email. Answers given via Piazza will help you as well as other students. Thus, emails will always have a lower priority for response than Piazza questions. If you do email the professor or TA, please make a note of what section you are in. This helps us identify you in courseworks faster.
</details>

**Q**: Create a function to return an HTML string. The HTML should contain three paragraphs, one photo, and 3 link elements. The paragraphs and link elements must contain content. The links must have a class of "large". The HTML must contain a title element and all necessary container tags.

[1 point]

In [ ]:
def get_html() -> str:
    html = "<!doctype html> <html lang=en> <head><meta charset=utf-8><title>end2122</title></head><body><p>Potatoes</p> <p>Are 2</p> <p>Potatoes 3</p> <img src='https://d2lnr5mha7bycj.cloudfront.net/product-image/file/large_6a3d718e-03ce-4af9-a006-19941820d02a.jpg'</img><class='large' href='https://www.google.com/search?q=potato'>potato</a><class='large' href='https://www.google.com/search?q=fries>fries'</a><class='large' href='https://www.google.com/search?q=sweet+potato'>sweet potato</a></body></html>"
    return html

In [ ]:
import bs4

soup = bs4.BeautifulSoup(get_html(), 'html.parser')

**Q**: Review the HTML for the front page of http://www.nytimes.com/. Count the number of links on the page. Don't do this by hand. Implement a function that takes the HTML for the page and returns the number of links on the page. 

[1 point]

In [2]:
from bs4 import BeautifulSoup as bs
import requests
response = requests.get('https://www.nytimes.com/')
results_page = bs(response.content, 'html.parser')
data = results_page.find_all('a')
links = list()
potato = 0
for tag in data:
    try:
        links.append(tag.get('href'))
    except:
        print('Error: '+tag)
        break
for i in links:
    potato+=1

potato

224

In [ ]:
import bs4

def count_links(page_source: bytes) -> int:
    from bs4 import BeautifulSoup as bs
    import requests
    response = requests.get('https://www.nytimes.com/')
    results_page = bs(response.content, 'html.parser')
    data = results_page.find_all('a')
    links = list()
    potato = 0
    for tag in data:
        try:
            links.append(tag.get('href'))
        except:
            print('Error: '+tag)
            break
    for i in links:
        potato+=1
    
    return potato

**Q**: Provided along with this homework is an HTML table of rental prices and dates of contract start from a real-estate website. Write three functions (the signatures of which are below) that will parse the rentals.html file for the following rental information: `(date, price, beds, baths, area)`. The `parse_html` function should return a list of tuples, each tuple containing information in the following form: 

Tuple order: `(date, price, beds, baths, area)`

- date: int - unix timestamp of activity. Accurate to the second.
- price: int - price of rental. Accurate to the dollar
- beds: int - number of bedrooms
- baths: int - number of bathrooms
- area: int - number of sq. ft. in rental (provide as available). 


The resultant list produced by `parse_html` should be ordered in the same way rows are ordered in the HTML table.

[3 points]

In [ ]:
import re

import bs4


def parse_html(html: bytes) -> list:
    soup = bs4.BeautifulSoup(html,'html.parser')
    x = soup.find_all('tbody',class_='listing_block loaded')[0].find_all('tr')
    potato = list()
    for row in x:
        m = parse_tr(row)
        potato.append(m)
    return potato


def parse_tr(row: bs4.element.Tag) -> tuple:
    """
    Return a tuple of (date, price, beds, baths, area) for the given row of data.

    This function should be used by parse_html.
    """
    date = parse_leading_number(row.find_all('td')[0].get('data-sort-value'))
    price = parse_leading_number(row.find_all('td')[2].get('data-sort-value'))
    beds = parse_leading_number(row.find_all('td')[3].get('data-sort-value') + ' beds')
    bath = parse_leading_number(row.find_all('td')[4].get('data-sort-value') + ' bath')
    area = parse_leading_number(re.search(r'\d+',row.find_all('td')[5].get_text()).group() + ' ft²')
    chk = (int(date),int(price),int(beds),int(bath),int(area))
    return chk
    
    
def parse_leading_number(string: str) -> str:
    """
    This function should be used by parse_tr.
    """
    mid = re.search(r'\d+',string).group()
    return mid

In [ ]:
### BEGIN TESTS
assert parse_leading_number('0 ft²') == '0'
### END TESTS

In [ ]:
### BEGIN TESTS
assert parse_leading_number('1 bath') == '1'
### END TESTS

In [ ]:
### BEGIN TESTS
assert parse_leading_number('2 beds') == '2'
### END TESTS

In [ ]:
### BEGIN TESTS
assert parse_leading_number('710 ft²') == '710'
### END TESTS

In [ ]:
### BEGIN TESTS
with open('rentals.html', 'rb') as fp:
    source = fp.read()
### END TESTS

In [ ]:
### BEGIN TESTS
assert len(parse_html(source)) == 57
### END TESTS

In [ ]:
# A graph to view your parsed results. This cell is ungraded and just for fun. 
import datetime

from bokeh.plotting import figure, output_notebook, show

output_notebook()

rows = parse_html(source)
x = [datetime.datetime.fromtimestamp(r[0]) for r in rows if r[2] == 1]
y = [r[1] for r in rows if r[2] == 1]

p = figure(
    plot_width=900, 
    plot_height=400,
    x_axis_type='datetime',
)
p.line(x, y, line_width=2)
p.xaxis.axis_label = 'Rental Contract Signed (Date)'
p.yaxis.axis_label = 'Rental Price ($)'
show(p)